# Creating a model and training it for generating and clustering music

In [ ]:
#TODO: Create models: VAE, WAE, U-NET
#Save model weights
#Evaluate in this file?

In [ ]:
#libraries
import deeplay as dl
import deeptrack as dt
import torch
import os
from PIL import Image, ImageOps
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Preprocess data
data_dir = os.path.expanduser("./trainImages")

#Load image files using ImageFolder
trainFiles = dt.sources.ImageFolder(root=data_dir)

print(f"Number of train images: {len(trainFiles)}")

In [ ]:
#Create image pipeline
class CropWidth:
    def __init__(self, target_width):
        self.target_width = target_width

    def __call__(self, x: torch.Tensor):
        # assuming input shape [C, H, W]
        return x[..., :self.target_width]
    
image_pip = (dt.LoadImage(trainFiles.path) >> dt.NormalizeMinMax()
             >> dt.MoveAxis(2, 0) >> dt.pytorch.ToTensor(dtype=torch.float) >> CropWidth(2560
             ))

In [ ]:
img_tensor = image_pip(trainFiles.path[0])
print(f"The size of each image is: {img_tensor.shape}")

In [ ]:
#VAE
from deeplay import AdamW
import torch
torch.cuda.empty_cache()
#Dont use reduction="sum" gradients becomes giant and memory issues are made worse. //BD
vae = dl.VariationalAutoEncoder(input_size=(256,2560),
    latent_dim=10, channels=[64, 128, 256, 512],
    reconstruction_loss=torch.nn.BCELoss(reduction="mean"), beta=1, optimizer=AdamW(lr=1e-3)
).create()

print(vae)

In [ ]:
#Giovanni used image_pip & image_pip here, do not really know why // L-Thor
#I think it is because one is used for the mean and one is used for the variance //BD
import dataset
train_dataset = dataset.Dataset(image_pip & image_pip, inputs=trainFiles)
print(train_dataset)
train_loader = dl.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=15)

In [ ]:
#NOTE: Training works, no errors, but it is VERY slow and my computer semi-freezes when I do it, perhaps something is wrong,
#or my computer is trash or maybe we just have to thug it out // L-Thor
#Memory issues, it seems to not properly reuse memory after each batch. Larger batches uses less memory which makes no sense to something
# s definetly wrong //BD
vae_trainer = dl.Trainer(max_epochs=10, accelerator="auto")
vae_trainer.fit(vae, train_loader)

### Plotting the training progress and saving the model

In [ ]:
import torch

vae_trainer.print()
torch.save(vae.state_dict(), "vae.pth")  #Saving the model.
#vae.load_state_dict(torch.load("vae.pth"))  // Use this to load the model! //BD

In [ ]:
#WAE, let us just focus on the VAE for now, later we can use this code
wae = dl.WassersteinAutoEncoder(
    channels=[32, 64, 128], latent_dim=20,
    reconstruction_loss=torch.nn.MSELoss(reduction="mean"),
).create()

#print(wae)

In [ ]:
#Example on how to generate music, just random noice now but with proper training and clustering we could try to sample things adjacent to a genre //BD

import torch
from Image2Sound import Image2Sound, SaveAudio
from torchvision.utils import save_image

vae.eval()  
n_samples = 1 
latent_dim = 20  
#Had to add a scalar to make the volume higher, think because of the normalization in the pipeline //BD
z = 255*torch.randn(n_samples, latent_dim).to(next(vae.parameters()).device)
print(z.shape)

with torch.no_grad():
    generated_image = vae.decode(z)
    #TODO: Change the Image2Sound so we don't have to define conf again here when generating, should be a simple enough fix. Alternatively just make a file holding the class
    #with a proper init //BD
    class conf:
        sampling_rate = 44100
        duration = 30
        hop_length = 694
        fmin = 20
        fmax = sampling_rate // 2
        n_mels = 128
        n_fft = n_mels * 20
        samples = sampling_rate * duration
    save_image(generated_image, 'generated_sample.jpg')
    audio = Image2Sound('generated_sample.jpg', conf)
    SaveAudio(audio,os.getcwd(),"testing.mp3")
